<a href="https://colab.research.google.com/github/BrenoAV/ChatBot_Pizzaria_Super_Legal/blob/main/ChatBot_Pizzaria_Super_Legal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q -U google-generativeai

In [15]:
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

GEMINI_KEY =  userdata.get("GEMINI_KEY")

genai.configure(api_key=GEMINI_KEY)

# Funções Auxiliares

In [62]:
def embed_fn(title: str,
             content: str) -> list[float]:
    return genai.embed_content(
        model="models/embedding-001",
        content=content,
        title=title,
        task_type="RETRIEVAL_DOCUMENT"
    )["embedding"]

def search_content(content: str,
                   df: pd.DataFrame)
    embedding_query = genai.embed_content(
        model="models/embedding-001",
        content=content,
        task_type="RETRIEVAL_QUERY"
    )["embedding"]



Lista todos os modelos para fazer "embedding" dos textos

In [7]:
for m in genai.list_models():
    if "embedContent" in m.supported_generation_methods:
        print(m.name)

models/embedding-001
models/text-embedding-004


Lista todos os modelos de geração de conteúdo

In [8]:
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


# Documentos

In [43]:
doc_pizzas_tradicionais = {
    "title": "Pizzas Tradicionais",
    "content" :
"""Preços das pizzas tradicionais:
  - R$ 20,00 (Pequena - P)
  - R$ 30,00 (Média - M)
  - R$ 40,00 (Grande - G)

01. Mussarela - Molho, mussarela, tomate, orégano e azeitona.
02. Dois queijos - Molho, mussarela, catupiry, orégano e azeitona.
03. Frango - Molho, frango, mussarela, milho, orégano e azeitona.
04. Vegetariana - Molho, milho, ervilha, cebola, tomate, palmito, brócolis, mussarela, orégano e azeitona.
05. Nordestina - Molho, carne cebola, mussarela, orégano e azeitona.
06. Calabresa - Molho, calabresa, bacon, orégano e azeitona.
07. Portuguesa - Molho, presunto, cebola, ervilha, ovos, mussarela, orégano e azeitona.
08. Baiana - Molho, calabresa, ovos, pimenta, cebola, mussarela, orégano e azeitona.
"""
}

In [44]:
doc_pizzas_especiais = {
    "title": "Pizzas Especiais",
    "content" :
"""Preços das pizzas especiais:
  - R$ 25,00 (Pequena - P)
  - R$ 35,00 (Média - M)
  - R$ 45,00 (Grande - G)

01. Camarão - Molho, camarão, mussarela, catupiry, orégano e azeitona.
02. Peru - Molho, peito de peru, mussarela, catupiry, orégano e azeitona.
03. Brasileira - Molho, peito de peru, parmesão, creme cheese, mussarela, bacon, orégano e azeitona.
"""
}

In [45]:
doc_pizzas_doces = {
    "title": "Pizzas Doces",
    "content":
"""Preços das pizzas especiais:
  - R$ 25,00 (Pequena - P)
  - R$ 35,00 (Média - M)
  - R$ 45,00 (Grande - G)

01. Sonho de Valsa - Bombons, creme de leite, leite condensado e cereja.
02. Brigadeiro - Chocolate, granulado, leite condensado e cereja.
03. Chocovalsa - Chocolate, bombons, leite condensado, creme de leite e cereja.
"""
}

In [46]:
doc_bebidas = {
    "title": "Bebidas",
    "content":
"""01. Água (R$ 2,00)
02. Refrigerante Lata (R$ 5,00)
03. Refrigerante 1L (R$ 8,00)
04. Refrigerante 2L (R$ 12,00)
05. Suco 600 mL (R$ 5,00)
"""
}

In [47]:
documentos = [
    doc_pizzas_tradicionais,
    doc_pizzas_especiais,
    doc_pizzas_doces,
    doc_bebidas
]

In [48]:
pizzaria_df = pd.DataFrame(documentos)
pizzaria_df.columns = ["title", "content"]
pizzaria_df.head()

,title,content
0,Pizzas Tradicionais,"Preços das pizzas tradicionais: \n - R$ 20,00..."
1,Pizzas Especiais,"Preços das pizzas especiais:\n - R$ 25,00 (Pe..."
2,Pizzas Doces,"Preços das pizzas especiais:\n - R$ 25,00 (Pe..."
3,Bebidas,"01. Água (R$ 2,00)\n02. Refrigerante Lata (R$ ..."


In [76]:
pizzaria_df["embedding"] = pizzaria_df.apply(
    lambda row: embed_fn(row["title"], row["content"]), axis=1
)
pizzaria_df.head()

,title,content,embedding
0,Pizzas Tradicionais,"Preços das pizzas tradicionais: \n - R$ 20,00...","[0.028574573, -0.050830718, 0.002385603, 0.004..."
1,Pizzas Especiais,"Preços das pizzas especiais:\n - R$ 25,00 (Pe...","[0.042846315, -0.05455409, -0.017772857, -0.01..."
2,Pizzas Doces,"Preços das pizzas especiais:\n - R$ 25,00 (Pe...","[0.023673648, -0.01095788, -0.030276284, -0.02..."
3,Bebidas,"01. Água (R$ 2,00)\n02. Refrigerante Lata (R$ ...","[0.0009966245, -0.011158399, -0.044617806, -0...."
